In [1]:
import os
import sys
import polars as pl
import plotly.express as px
import numpy as np
import polars.selectors as cs

PROJECT_PATH = os.path.abspath(os.path.join("..", ".."))
PIPELINE_PATH = os.path.join(PROJECT_PATH, "pipeline")
DATA_DIRECTORY = os.path.join(PROJECT_PATH, "data")

if PIPELINE_PATH not in sys.path:
    sys.path.append(PIPELINE_PATH)

In [2]:
column = "gmp343_edge_corrected"

In [3]:
df = pl.read_parquet(os.path.join(DATA_DIRECTORY, "output", "side-by-side", "2025_L1_1_min_sbs_period_acropolis.parquet")) \
    .sort("datetime") \
    .with_columns(diff = pl.col(column) - pl.col("picarro_corrected")) \
    .group_by_dynamic("datetime", every='1h', group_by=["system_id", "system_name"]) \
    .agg([
            cs.numeric().mean(),
            pl.col(column).std().alias("gmp343_corrected_std"),
            pl.col(column).var().alias("gmp343_corrected_var"),
            pl.col(column).count().alias("gmp343_corrected_count"),
        ]) \
    .sort("system_id")

In [4]:
df_sbs = df.filter(pl.col("diff").is_not_nan()) \
    .group_by(["system_id", "system_name"]) \
    .agg([
            (pl.col("diff").mean()).alias("mean_full_deployment"),
            (np.abs(pl.col("diff"))).mean().alias("mae_full_deployment"),
            (pl.col("diff").pow(2).mean()).alias("mse_full_deployment"),
            (pl.col("diff").pow(2).mean().sqrt()).alias("rmse_full_deployment")
        ]) \
    .sort("system_id")

In [5]:
fig = px.scatter(df, x="datetime", y="gmp343_corrected_count", color = "system_name")
fig.show()

In [6]:
df_daily_mean = df.sort("datetime") \
    .with_columns(pl.col("datetime").dt.date().alias("date")) \
    .group_by(["date", "system_name", "system_id"]) \
    .agg([
        (pl.col("diff").mean()).alias("daily_mean"),
        (pl.col("diff").median()).alias("daily_median")
        ]) \
    .sort("system_id")
    
fig = px.scatter(df_daily_mean, x="date", y="daily_mean", color = "system_name")
fig.show()
fig = px.scatter(df_daily_mean, x="date", y="daily_median", color = "system_name")
fig.show()

In [7]:
fig = px.scatter(df, x="datetime", y="diff", color = "system_name")
fig.show()
fig = px.scatter(df_sbs, x="system_name", y="mean_full_deployment")
fig.show()
fig = px.scatter(df_sbs, x="system_name", y="mae_full_deployment")
fig.show()
fig = px.scatter(df_sbs, x="system_name", y="rmse_full_deployment")
fig.show()

In [8]:
df_plot = df.sort("datetime") \
    .group_by(["system_name", "system_id"]) \
    .agg((pl.col("gmp343_corrected").count() / 24).cast(int).alias("sbs_days")) \
    .join(df_sbs, on="system_id") \
    .drop("^.*_right$") \
    .sort("system_id")

fig = px.scatter(df_plot, x="system_name", y="sbs_days")
fig.show()